<a href="https://colab.research.google.com/github/Lomzem/LomzemStockQuant/blob/main/LomzemStocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os
import pandas as pd
import datetime
import sys

wanted_sheet_path = '/content/drive/MyDrive/LomzemStockCharts/stocks_to_obtain.xlsx'
raw_stock_data_path = '/content/drive/MyDrive/LomzemStockCharts/rawstockdata'

def get_stock_data(date, ticker):
	api_url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=50000&apiKey=IyEWxobQmnBJh0QwQeNOWH7QggrJGFjp'
	try:
		data = requests.get(api_url).json()
		stock_df = pd.DataFrame(data['results'])
		stock_df.rename(columns = {'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 'v': 'Volume', 't': 'Time'}, inplace=True)
		stock_df.insert(0, 'Ticker', ticker)
		stock_df.insert(1, 'FTime', stock_df.apply(lambda row: pd.to_datetime(row['Time']-1.44e+7, unit='ms'), axis=1))
		stock_df.to_excel(f'{raw_stock_data_path}/{ticker}-{date}.xlsx', index=False)
		print(f'Got {raw_stock_data_path}/{ticker}-{date}.xlsx')
	except KeyError:
		print('Wait a little longer')
		print(date, ticker)
		sys.exit()

def read_wanted_sheet():
	df = pd.read_excel(wanted_sheet_path)
	for index, row in df.iterrows():
		files = [file for file in os.listdir(raw_stock_data_path)]
		if not (f'{row["Ticker"]}-{row["Date"].date()}.xlsx') in files:
			if row['Date'].date() < datetime.date.today()-datetime.timedelta(days=1):
				get_stock_data(row['Date'].date(), row['Ticker'].upper())

read_wanted_sheet()

In [ ]:
import pandas as pd
import os

cum_stock_data = pd.DataFrame()

for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata'):
	df = pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{file}')
	cum_stock_data = pd.concat([cum_stock_data, df])

print(cum_stock_data)
cum_stock_data.to_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx', index=False)

    Ticker               FTime   Volume      vw  Open  Close  High   Low  \
0     VERU 2022-04-11 06:30:00   7448.0  5.1801  4.91   5.47  5.48  4.55   
1     VERU 2022-04-11 06:31:00   6869.0  5.3946  5.20   5.45  5.93  5.16   
2     VERU 2022-04-11 06:32:00  49308.0  6.0368  5.42   5.72  6.53  5.42   
3     VERU 2022-04-11 06:33:00  38057.0  5.6815  5.73   5.71  5.90  5.50   
4     VERU 2022-04-11 06:34:00  29283.0  5.6577  5.75   5.70  5.83  5.51   
..     ...                 ...      ...     ...   ...    ...   ...   ...   
169   AVDL 2022-04-22 15:58:00   1868.0  6.6009  6.61   6.61  6.61  6.59   
170   AVDL 2022-04-22 15:59:00   4768.0  6.6020  6.61   6.60  6.61  6.59   
171   AVDL 2022-04-22 16:00:00   3621.0  6.5800  6.58   6.58  6.58  6.58   
172   AVDL 2022-04-22 18:13:00   2500.0  6.2600  6.26   6.26  6.26  6.26   
173   AVDL 2022-04-22 18:14:00    500.0  6.2600  6.26   6.26  6.26  6.26   

              Time    n   a  op  
0    1649673000000   55 NaN NaN  
1    1649673060000 

In [ ]:
import requests
import os
import pandas as pd
import datetime

df = pd.read_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx')
ddf = pd.DataFrame()

ddf = pd.concat([ddf, df[['Ticker', 'FTime']]])
ddf['FTime'] = ddf['FTime'].dt.date

# df.loc[192]
# Get Open
t_df = df.loc[df['FTime'].dt.time==datetime.time(9,30), ['Ticker', 'FTime', 'Open']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Close
t_df = df.loc[df['FTime'].dt.time==datetime.time(15,59), ['Ticker', 'FTime', 'Close']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get High
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'High']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['High'].transform(max) == t_df['High']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Low
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'Low']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['Low'].transform(min) == t_df['Low']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Prev Close
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['PrevClose'] = ddf.groupby('Ticker').shift(1)['Close']

# Get Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<=datetime.time(20,0))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df = t_df.groupby(['Ticker', 'FTime'])['Volume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Premarket Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<datetime.time(9,30))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df.rename(columns={'Volume': 'PVolume'}, inplace=True)
t_df = t_df.groupby(['Ticker', 'FTime'])['PVolume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Gap %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['Gap%'] = (ddf['Open'] / ddf['PrevClose'] - 1) * 100

# Get Premarket High
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,00)) & (df['FTime'].dt.time<datetime.time(9,30))]
t_df = t_df[['Ticker', 'FTime', 'High']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df.rename(columns={'High': 'PHigh'}, inplace=True)
idx = t_df.groupby(['Ticker', 'FTime'])['PHigh'].transform(max) == t_df['PHigh']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])
ddf.insert(5, 'PHigh', ddf.pop('PHigh'))

# Get Change From Open %

ddf['ChangeFromOpen%'] = (ddf['Close']/ddf['Open']-1)*100

# Get HOD - PrevClose %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['HOD-PrevClose%'] = (ddf['High'] / ddf['PrevClose'] - 1) * 100

# Get HOD - PrevHOD %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['HOD-PrevHOD%'] = (ddf['High'] / ddf.groupby('Ticker').shift(1)['High'] - 1) * 100

# Get Range %
ddf['Range%'] = (ddf['High'] / ddf['Low'] - 1) * 100

# Get Volume Ratio
ddf['Vol Ratio'] = ddf['Volume']/ddf['PVolume']

ddf.rename(columns={'FTime': 'Date'}, inplace=True)

ddf['closedRed'] = ddf['Close'] < ddf['Open']

### PATTERN: Overextended Gap Down

def overextended_gap_down():
  ddf['isOEGD'] = False
  prev_change_from_open = ddf.groupby('Ticker').shift(1)['ChangeFromOpen%']
  ddf.loc[(prev_change_from_open > 30) & (ddf['Gap%']<0), ['isOEGD']] = True

### PATTERN: Gap Up Short

def gap_up_short():
  ddf['isGUS'] = False
  ddf.loc[(ddf['Gap%'] > 35) & (ddf['closedRed']==True) & (ddf['Low'] / ddf['Open']-1 <= -0.1), ['isGUS']] = True

overextended_gap_down()
gap_up_short()

ddf['High-Open%'] = (ddf['High']/ddf['Open']-1)*100

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
display(ddf.loc[(ddf['Low']/ddf['Open']-1<-.35) & (ddf['Close']/ddf['Low']-1>.2)])

In [ ]:
import os
import pandas as pd

def search_for_duplicates_in_wanted():
    wanted_sheet_path = '/content/drive/MyDrive/LomzemStockCharts/stocks_to_obtain.xlsx'
    raw_stock_data_path = '/content/drive/MyDrive/LomzemStockCharts/rawstockdata'
    df = pd.read_excel(wanted_sheet_path)
    df.drop_duplicates(inplace = True)
    df.dropna(inplace=True)
    df.sort_values(['Ticker'])
    df['Date'] = df['Date'].dt.date
    df.to_excel(wanted_sheet_path, index = False)

search_for_duplicates_in_wanted()

In [21]:
#PLOTLY TEST

import pandas as pd
import plotly.graph_objects as go
import datetime

pd.options.display.max_rows = 60

def make_plotly_graph(ticker, date, show_d0=False, show_d2=False):
    cdf = pd.DataFrame()
    files = [file for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata')]
    files = sorted(files)

    # Just Day 1
    if show_d0 == False and show_d2 == False:
        cdf = pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{ticker}-{date}.xlsx')
        fig = go.Figure(data = [go.Candlestick(x=cdf['FTime'], open = cdf['Open'], high = cdf['High'], close=cdf['Close'], low=cdf['Low'])])
        fig.update_layout(xaxis_rangeslider_visible=False, template='plotly_dark', title=f'{ticker} on {date} (1 min)')

        # Add Premarket Rectangle for Day 1
        fig.add_vrect(x0 = cdf.iloc[0]['FTime'], x1 = datetime.datetime.combine(date, datetime.time(9, 29)), fillcolor='#3f3e3f', opacity=0.44, line_width=0)
        fig.add_hline(y=(ddf.iloc[ddf.index[(ddf['Date']==date) & (ddf['Ticker']==ticker)]-1]['Close']).values[0], line_color='#F6FEAA', line_dash='dot')
        fig.show()
    
    # Day 0 + Day 1
    elif show_d0 == True and showd2 == False:
        for file in files:
            if file.endswith(f'{ticker}-{date}.xlsx'):
                file_index = files.index(file)
                cdf = pd.concat([cdf, pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{files[file_index-1]}')]) # Read Yesterday
                cdf = pd.concat([cdf, pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{files[file_index]}')]) # Read Today

        cdf.sort_values(['Ticker', 'FTime'], inplace=True)
        fig = go.Figure(data = [go.Candlestick(x=cdf['FTime'], open = cdf['Open'], high = cdf['High'], close=cdf['Close'], low=cdf['Low'])])
        fig.update_layout(xaxis_rangeslider_visible=False, template='plotly_dark', title=f'{ticker} on {date} (1 min)')
        day0 = datetime.date.fromisoformat('-'.join(files[file_index-1][:-5].split('-')[1:4]))
        day1 = date

        start_omit_time = datetime.datetime.combine(day0, datetime.time(20, 1))
        end_omit_time = datetime.datetime.combine(day1, datetime.time(3, 59))

        fig.update_xaxes(rangebreaks=[dict(bounds=[start_omit_time, end_omit_time])])
        fig.add_vrect(x0 = cdf.iloc[0]['FTime'], x1 = datetime.datetime.combine(day0, datetime.time(9, 29)), fillcolor='#3f3e3f', opacity=0.44, line_width=0) # Day 0 Premarket Rectangle
        fig.add_vrect(x0 = datetime.datetime.combine(day0, datetime.time(16, 0)), x1 = datetime.datetime.combine(day1, datetime.time(9, 29)), fillcolor='#3f3e3f', opacity=0.44, line_width=0) # Day 0 Aftermarket + Day 1 Premarket
        fig.add_vrect(x0 = datetime.datetime.combine(day1, datetime.time(16, 0)), x1 = cdf.iloc[-1]['FTime'], fillcolor='#3f3e3f', opacity=0.44, line_width=0) # Day 1 Aftermarket
        fig.add_hline(y=(ddf.iloc[ddf.index[(ddf['Date']==date) & (ddf['Ticker']==ticker)]-1]['Close']).values[0], line_color='#F6FEAA', line_dash='dot')
        fig.show()

    # Day 1 + Day 2
    elif show_d0 == False and show_d2 == True:
        for file in files:
            if file.endswith(f'{ticker}-{date}.xlsx'):
                file_index = files.index(file)
                cdf = pd.concat([cdf, pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{files[file_index]}')]) # Read Today
                cdf = pd.concat([cdf, pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{files[file_index+1]}')]) # Read Tomorrow

        cdf.sort_values(['Ticker', 'FTime'], inplace=True)
        fig = go.Figure(data = [go.Candlestick(x=cdf['FTime'], open = cdf['Open'], high = cdf['High'], close=cdf['Close'], low=cdf['Low'])])
        fig.update_layout(xaxis_rangeslider_visible=False, template='plotly_dark', title=f'{ticker} on {date} (1 min)')
        day1 = date
        day2 = datetime.date.fromisoformat('-'.join(files[file_index+1][:-5].split('-')[1:4]))

        start_omit_time = datetime.datetime.combine(day1, datetime.time(20, 1))
        end_omit_time = datetime.datetime.combine(day2, datetime.time(3, 59))

        fig.update_xaxes(rangebreaks=[dict(bounds=[start_omit_time, end_omit_time])])
        fig.add_vrect(x0 = cdf.iloc[0]['FTime'], x1 = datetime.datetime.combine(day1, datetime.time(9, 29)), fillcolor='#3f3e3f', opacity=0.44, line_width=0) # Day 1 Premarket Rectangle
        fig.add_vrect(x0 = datetime.datetime.combine(day1, datetime.time(16, 0)), x1 = datetime.datetime.combine(day2, datetime.time(9, 29)), fillcolor='#3f3e3f', opacity=0.44, line_width=0) # Day 1 Aftermarket + Day 2 Premarket
        fig.add_vrect(x0 = datetime.datetime.combine(day2, datetime.time(16, 0)), x1 = cdf.iloc[-1]['FTime'], fillcolor='#3f3e3f', opacity=0.44, line_width=0) # Day 2 Aftermarket
        fig.add_hline(y=(ddf.iloc[ddf.index[(ddf['Date']==date) & (ddf['Ticker']==ticker)]]['Close']).values[0], line_color='#F6FEAA', line_dash='dot')
        fig.show()

for index, row in ddf.sort_values('Date', ascending=False).iterrows():
    if row['Gap%']>35:
        make_plotly_graph(row['Ticker'], row['Date'], show_d2=True)